<a href="https://colab.research.google.com/github/teatime77/xbrl-reader/blob/master/notebook/preprocess.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### データをダウンロードします。

In [0]:
! wget http://lkzf.info/xbrl/data/2020-04-08/summary-0.csv
! wget http://lkzf.info/xbrl/data/2020-04-08/summary-1.csv
! wget http://lkzf.info/xbrl/data/2020-04-08/summary-2.csv

##### 会社情報をダウンロードします。

In [0]:
! wget http://lkzf.info/xbrl/data/EdinetcodeDlInfo.csv

#### <font color="red">以下の中から予測したい項目のコメントをはずしてください。</font>

In [0]:
target = '売上高'
# target = '営業利益'
# target = '経常利益'
# target = '税引前純利益'

### 必要なライブラリをインポートします。

In [0]:
import pandas as pd
import numpy as np

### CSVファイルの読み込みの関数

In [0]:
def my_read_csv(filename, date_columns=[]):
    """ How to force pandas read_csv to use float32 for all float columns?
    https://stackoverflow.com/questions/30494569/how-to-force-pandas-read-csv-to-use-float32-for-all-float-columns
    """

    # 最初の100行のみ試しに読んでみる。
    df_test = pd.read_csv(filename, nrows=100, parse_dates=date_columns)

    # float64の列の名前
    float_cols = [c for c in df_test if df_test[c].dtype == "float64"]

    # 型をfloat32として宣言する。
    float32_cols = {c: np.float32 for c in float_cols}

    # 型の宣言を使ってCSVファイルを読み直す。
    return pd.read_csv(filename, engine='c', dtype=float32_cols, parse_dates=date_columns)

### 決算書のCSVファイルを読み込みます。

In [0]:
print("read 提出日時点 ...")
df提出日時点 = my_read_csv("summary-0.csv", [ '会計期間終了日' ])
print("read 時点 ...")
df時点 = my_read_csv("summary-1.csv")
print("read 期間 ...")
df期間 = my_read_csv("summary-2.csv")

### 有価証券報告書のみを抜き出します。

In [0]:
df提出日時点 = df提出日時点[ df提出日時点['報告書略号'] == "asr" ]
df時点       = df時点[ df時点['報告書略号'] == "asr" ]
df期間       = df期間[ df期間['報告書略号'] == "asr" ]

for tbl in [ df提出日時点, df時点, df期間 ]:
    tbl.drop('報告書略号', axis=1, inplace=True)

### df時点とdf期間の列で、df提出日時点の列と重複があれば取り除きます。

In [0]:
除外columns = ['index', 'EDINETコード', '会計期間終了日', 'コンテキスト' ]

for tbl in [ df時点, df期間 ]:
    
    # 重複する列の名前のリスト
    dup_columns = [x for x in tbl.columns if x in df提出日時点 and not x in 除外columns]

    if len(dup_columns) != 0:
        # 重複がある場合

        print("重複", dup_columns)
        tbl.drop(dup_columns, axis=1, inplace=True)

### df時点とdf期間で同じ名前の列があれば、名前に「-時点」や「-期間」を付加します。

In [0]:
v = [x for x in df時点.columns if not x in 除外columns]
for name in v:
    if name in df期間.columns:
        print('%d %d %s' % (df時点[name].notnull().sum(), df期間[name].notnull().sum(), name))
        df時点.rename(columns={ name: name+'-時点' }, errors='raise', inplace=True)                    
        df期間.rename(columns={ name: name+'-期間' }, errors='raise', inplace=True)

### 内容が同じで名前が違う2つの列を1つに統合します。

In [0]:
def unify_columns(tbl, alias, name):
    diff = 0

    # 別名と本名の列のそれぞれの値に対し
    for idx, (alias_val, val) in enumerate(zip(tbl[alias], tbl[name])):
        if not alias_val is None:
            # 別名の値がある場合

            if val is None:
                # 本名の値がない場合

                # 別名の値を本名の値とする。
                tbl[name].iloc[idx] = alias_val
            else:
                # 本名の値がある場合

                if val != 0:

                    # 別名と本名の値の違いを調べる。
                    diff = max(diff, abs((alias_val - val)/val) )

    print('%.5f %s' % (diff, alias))

In [0]:
for suffix in [ '（IFRS）', '、経営指標等', '（平成26年3月28日財規等改正後）', '又は売上総損失（△）' ]:
    for tbl in [df時点, df期間]:
        drop_names = []
        for name in tbl.columns:
            
            if  suffix in name:
                # 名前に接尾辞が含まれる場合

                # 名前から接尾辞を取り除く。
                short_name = name.replace(suffix, '')
                
                if short_name in tbl.columns:
                    # 短い名前の列がある場合

                    # 短い名前の列に統合する。
                    unify_columns(tbl, name, short_name)

                    # 長い名前の列は削除する。
                    drop_names.append(name)

                else:
                    # 短い名前の列がない場合

                    if suffix != '（IFRS）':

                        # 長い名前を短い名前に変える。
                        print('短縮 : %s -> %s' % (name, short_name))
                        tbl.rename(columns={ name:short_name }, errors='raise', inplace=True)                    

        # 長い名前の列を削除する。
        tbl.drop(drop_names, axis=1, inplace=True)

### 長い項目名を短くします。

In [0]:
df期間.columns

In [0]:
df時点 = df時点.rename(columns={ 
    '１株当たり純資産額': '１株当たり純資産',
    '平均年齢（年）'    : '平均年齢',
    '平均勤続年数（年）': '平均勤続年数',
})

df期間 = df期間.rename(columns={ 
    # '売上総利益又は売上総損失（△）'            :'売上総利益', 
    '経常利益又は経常損失（△）'                : '経常利益', 
    '営業利益又は営業損失（△）'                : '営業利益',
    '当期純利益又は当期純損失（△）'            :'純利益', 
    '税引前当期純利益又は税引前当期純損失（△）':'税引前純利益', 
    '１株当たり当期純利益又は当期純損失（△）'  :'１株当たり純利益',
    '親会社株主に帰属する当期純利益又は親会社株主に帰属する当期純損失（△）': '親会社株主に帰属する純利益',
    '潜在株式調整後1株当たり当期純利益'         :'調整1株当たり純利益',
    '現金及び現金同等物の増減額（△は減少）'    :'現金及び現金同等物の増減' 
}, errors='raise')

### コンテキストごとに分離します。

In [0]:
df提出日時点 = df提出日時点.set_index(['EDINETコード', '会計期間終了日'])

tbls = []
for tbl, ctx in [ [df時点, '当期連結時点'], [df時点, '当期個別時点'], [df時点, '前期連結時点'], [df時点, '前期個別時点'], \
                  [df期間, '当期連結期間'], [df期間, '当期個別期間'], [df期間, '前期連結期間'], [df期間, '前期個別期間'] ]:

    tmp = tbl[tbl['コンテキスト'] == ctx]
    
    tmp = tmp.set_index(['EDINETコード', '会計期間終了日'])
    
    tmp = tmp.drop('コンテキスト', axis=1)

    tbls.append(tmp)

df当期連結時点, df当期個別時点, df前期連結時点, df前期個別時点, df当期連結期間, df当期個別期間, df前期連結期間, df前期個別期間 = tbls

assert len(df提出日時点) == len(df当期連結時点) == len(df当期個別時点) == len(df当期連結期間) == len(df当期個別期間) == len(df前期連結時点) == len(df前期個別時点) == len(df前期連結期間) == len(df前期個別期間)

### 会計終了時点と会計期間の表を連結します。

In [0]:
df当期連結 = pd.concat([ df当期連結時点, df当期連結期間 ], axis=1)
df当期個別 = pd.concat([ df当期個別時点, df当期個別期間 ], axis=1)
df前期連結 = pd.concat([ df前期連結時点, df前期連結期間 ], axis=1)
df前期個別 = pd.concat([ df前期個別時点, df前期個別期間 ], axis=1)

assert len(df当期連結) == len(df当期連結時点)  == len(df当期連結期間) == len(df当期個別) == len(df当期個別時点)  == len(df当期個別期間)
assert len(df前期連結) == len(df前期連結時点)  == len(df前期連結期間) == len(df前期個別) == len(df前期個別時点)  == len(df前期個別期間)

assert list(df当期連結.columns) == list(df当期個別.columns) == list(df前期連結.columns) == list(df前期個別.columns)

In [0]:
column_cnts = df当期連結.notnull().sum().items()
sorted_column_cnts = sorted(column_cnts, key=lambda x: x[1], reverse=True)

sorted_columns = [x[0] for x in sorted_column_cnts]

list(enumerate(sorted_column_cnts[:200]))

### 期首期末平均資産を計算します。

In [0]:
def set_assets_mean(tbl):
    # 期首期末平均資産
    mean_assets = np.ndarray(shape=(len(tbl),), dtype=np.float32)

    # EDINETコード
    edinet_codes = tbl.index.map(lambda x: x[0])
    
    prev_code = None
    prev_value = None

    # EDINETコードと資産に対し
    for i, [code, value] in enumerate(zip(edinet_codes, tbl['資産'])):
        if prev_code != code:
            # 会社の最初の場合

            # 前期がないので今期の資産を期首期末平均資産とする。
            mean_assets[i] = value

            prev_code = code
        else:
            # 会社の最初でない場合

            # 前期と今期の平均の資産を期首期末平均資産とする。
            mean_assets[i] = (prev_value + value) / 2

        # 前期の資産
        prev_value = value

    tbl['期首期末平均資産'] = mean_assets

for tbl in [ df当期連結, df当期個別, df前期連結, df前期個別 ]:
    set_assets_mean(tbl)

### 財務指標を計算します。

In [0]:
for tbl in [ df当期連結, df当期個別, df前期連結, df前期個別 ]:
    tbl['粗利益'] = tbl['売上高'] - tbl['売上原価']

    tbl['売上高総利益率']   = tbl['粗利益']   / tbl['売上高']
    tbl['売上高営業利益率'] = tbl['営業利益'] / tbl['売上高']
    tbl['売上高経常利益率'] = tbl['経常利益'] / tbl['売上高']

    tbl['売上高販管費率']   = tbl['販売費及び一般管理費'] / tbl['売上高']

    tbl['総資本回転率']     = tbl['売上高'] / tbl['資産']
    tbl['流動比率']         = tbl['流動資産'] / tbl['流動負債']

    tbl['総資産経常利益率'] = tbl['経常利益'] / tbl['期首期末平均資産']

    tbl['総資産純利益率']   = tbl['純利益']   / tbl['期首期末平均資産']

    tbl['総資産親会社株主に帰属する純利益率']   = tbl['親会社株主に帰属する純利益'] / tbl['期首期末平均資産']

    tbl['自己資本'] = tbl['株主資本'].fillna(0) + tbl['評価・換算差額等'].fillna(0)

    tbl['有利子負債'] = tbl['短期借入金'].fillna(0) + tbl['1年内返済予定の長期借入金'].fillna(0) + tbl['1年内償還予定の社債'].fillna(0) + tbl['長期借入金'].fillna(0) \
        + tbl['社債'].fillna(0) + tbl['転換社債型新株予約権付社債'].fillna(0) + tbl['コマーシャル・ペーパー'].fillna(0)

    tbl['負債比率'] = tbl['負債'] / tbl['自己資本']
    tbl['有利子負債比率'] = tbl['有利子負債']  / tbl['自己資本']

### 前期の列名に「前期」の接頭語を付けます。

In [0]:
for tbl in [ df前期連結, df前期個別 ]:
    tbl.rename(columns=dict((x, '前期' + x) for x in tbl.columns), inplace=True)

### 連結と個別のそれぞれの当期と前期の表を連結します。

In [0]:
df連結 = pd.concat([ df当期連結, df前期連結 ], axis=1)
df個別 = pd.concat([ df当期個別, df前期個別 ], axis=1)

assert len(df連結) == len(df当期連結) == len(df前期連結) == len(df個別) == len(df当期個別) == len(df前期個別)

### 当期の列名のリストを得ます。

In [0]:
assert list(df当期連結.columns) == list(df当期個別.columns)

num_columns = list(df当期連結.columns)

# 列名の重複がないことを確認
assert len(num_columns) == len(set(num_columns))

### 前期から当期への変化率を計算します。

In [0]:
for tbl in [ df連結, df個別 ]:
    for title in num_columns:
        tbl[title + '変化率'] = (tbl[title] - tbl['前期' + title]) / tbl['前期' + title]

### 前期のデータは以降で使わないので削除します。

In [0]:
drop_columns = ['前期' + x for x in num_columns]

df連結 = df連結.drop(drop_columns, axis=1)
df個別 = df個別.drop(drop_columns, axis=1)


### 個別の列名に「個別」の接頭語を付けます。

In [0]:
df個別.rename(columns=dict((x, '個別' + x) for x in df個別.columns), inplace=True)

### 連結と個別のデータを１つにまとめます。

In [0]:
df = pd.concat([ df連結, df個別 ], axis=1)

assert len(df) == len(df提出日時点)

# 列名の重複がないことを確認します。
assert len(df.columns) == len(set(df.columns))

#### 次期変化率を計算します。

In [0]:
# 予測する列の名前
y_column = target + '次期変化率'

# 次期変化率の配列をNaNで初期化します。
change_rate = np.array([np.nan] * len(df), dtype=np.float32)

# EDINETコード
edinet_codes = df.index.map(lambda x: x[0])

# 直前のEDINETコード
prev_code = None

# 直前の値
prev_value = None

for i, [code, value] in enumerate(zip(edinet_codes, df[target])):
    if prev_code != code:
        # 会社が変わった場合

        prev_code = code
    else:
        # 会社が同じ場合

        if prev_value != 0:
            # 直前の値が0でない場合

            # 次期変化率を計算します。
            change_rate[i - 1] = (value - prev_value) / prev_value

    prev_value = value

df[y_column] = change_rate

### 次期変化率が欠損値の行を削除します。

In [0]:
df = df.dropna(subset=[y_column])

### 比率以外の列は財務分析に使わないので削除します。

In [0]:
df = df.drop([x for x in df.columns if not x.endswith('率')], axis=1)

### 会社情報をセットします。

#### 会社情報のCSVを読み込みます。

In [0]:
df会社 = pd.read_csv("EdinetcodeDlInfo.csv", encoding='cp932', skiprows=[0])
df会社 = df会社.set_index('ＥＤＩＮＥＴコード')

#### 会社名から "株式会社" を取り除きます。

In [0]:
df会社['提出者名'] = df会社['提出者名'].apply(lambda x: x.replace('株式会社', '').strip())

#### 会社名と業種をセットします。

In [0]:
edinet_codes = df.index.map(lambda x: x[0])
df['会社名']  = edinet_codes.map(lambda x: df会社.loc[x]['提出者名'])
df['業種']    = edinet_codes.map(lambda x: df会社.loc[x]['提出者業種'])

In [0]:
df = df.reset_index()
df = df.set_index(['EDINETコード', '会社名', '会計期間終了日'])

## 出現頻度が高い列を残します。

### 出現頻度の高い順に列名を並べます。

In [0]:
# 有効値が多い列を得ます。
column_cnts = df.notnull().sum().items()
freq_column_cnts = sorted(column_cnts, key=lambda x: x[1], reverse=True)

list(enumerate(freq_column_cnts[:10]))

### 出現頻度の低い列は削除します。

In [0]:
drop_names = [x[0] for x in freq_column_cnts][200:]
df = df.drop(drop_names, axis=1)

## データを正規化します。

### 無限大はNaNに置換します。

In [0]:
for name in [x for x in df.columns if x != '業種']:
    df[name] = df[name].replace([np.inf, -np.inf], np.nan)

### 平均0、標準偏差1に正規化します。
### 欠損値は0で置き換えます。

In [0]:
def norm(v):
    # 平均
    mean = v.mean()

    # 標準偏差
    std  = v.std()

    assert not( np.isnan(mean) or np.isnan(std) or std == 0 )

    w = (v - mean) / std

    # 欠損値は0で置き換えます。
    return w.fillna(0)

In [0]:
for name in [x for x in df.columns if x != '業種']:
    df[name] = norm(df[name])

### 外れ値をクリップします。

In [0]:
nrow, ncol = (5, 4)

names = [x[0] for x in freq_column_cnts if x[0] != '業種']

for label in names[: nrow*ncol]:
    tmp = df[label]
    df[label] = tmp.clip(lower=tmp.quantile(0.05), upper=tmp.quantile(0.95))


## 相関係数が高い列を残します。

### 相関係数を計算します。

In [0]:
name_corrs = [ [name, np.corrcoef( np.stack([df[y_column].values, df[name].values]) )[0, 1] ] for name in df.columns if not name in ['業種', y_column] ]

# 欠損値は削除します。
name_corrs = [ x for x in name_corrs if not np.isnan(x[1])]

### 相関係数の絶対値の大きい順にソートします。

In [0]:
sorted_name_corrs = sorted(name_corrs, key=lambda x: abs(x[1]), reverse=True)
['%d  %.5f %s' % (i, x[1], x[0]) for i, x in enumerate(sorted_name_corrs[:50]) ]

### 相関係数が低い列を削除します。

## 結果を保存します。

In [0]:
sorted_names = [x[0] for x in sorted_name_corrs]
drop_names = sorted_names[50:] 
df = df.drop(drop_names, axis=1)


### 業種をダミー変数に変換します。

In [0]:
df = pd.get_dummies(df)

# '業種'の列がなくなったことを確認
assert not '業種' in df.columns

### pickleでファイルに書きます。

In [0]:
import pickle

data = { 'y_column': y_column, 'data_frame': df  }

if   y_column == '売上高次期変化率':
    file_name = 'preprocess-uriage.pickle'
elif y_column == '営業利益次期変化率':
    file_name = 'preprocess-eigyo.pickle'
elif y_column == '経常利益次期変化率':
    file_name = 'preprocess-keijo.pickle'
elif y_column == '税引前純利益次期変化率':
    file_name = 'preprocess-jun.pickle'
else:
    assert False

with open(file_name, 'wb') as f:
    pickle.dump(data, f)

#### <font color="red">パソコンにファイルをダウンロードする場合は以下のコメントをはずしてください。</font>

In [0]:
# from google.colab import files
# from pathlib import Path

# # pickleファイルに対し
# for path_obj in Path().glob('*.pickle'):
#     print(path_obj)

#     # ファイルをダウンロードします。
#     files.download(str(path_obj))